# Prepare data

In [2]:
import pandas as pd
columns = ['game id', 'move id', 'player id', 'column', 'winner']
data = pd.read_csv('c4-10k.csv', header=None, names=columns)

data.head(3)

,game id,move id,player id,column,winner
0,17,0,-1,3,-1
1,17,1,1,4,-1
2,17,2,-1,0,-1


In [3]:
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 0)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 1)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 2)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 3)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 4)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 5)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 6)]))

489
751
1213
2589
1870
1635
1453


In [7]:
column_names = []
for i in range(6*7):
    column_names.append("current_" + str(i))
column_names.append("player")
column_names.append("nextmove column")
board_df = pd.DataFrame(columns=column_names)
board_df.describe()

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_34,current_35,current_36,current_37,current_38,current_39,current_40,current_41,player,nextmove column
count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
import copy
def generate_boards():
    for game_id in data['game id'].unique():
        current_board = [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]
        for move_id in data.loc[data['game id'] == game_id]['move id'].unique():
            next_board = copy.deepcopy(current_board)
            
            # update next board
            column = data.loc[(data['game id'] == game_id) & (data['move id'] == move_id)]['column'].unique()[0].item()
            player = data.loc[(data['game id'] == game_id) & (data['move id'] == move_id)]['player id'].unique()[0].item()
            winner = data.loc[data['game id'] == game_id]['winner'].unique()[0].item()
            
            for row in range(6):
                if next_board[row][column] == 0:
                    next_board[row][column] = player
                    break
            
            # if next board move comes from a winning player keep data
            if player == winner:
                full_board = copy.deepcopy(current_board)
                full_board.append([player])
                full_board.append([column])
                flat_board = [item for sublist in full_board for item in sublist]
                
                board_df.loc[len(board_df)] = flat_board
            
            # set the new move as the next current move
            current_board = copy.deepcopy(next_board)

In [9]:
generate_boards()

In [10]:
board_df.head(5)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_34,current_35,current_36,current_37,current_38,current_39,current_40,current_41,player,nextmove column
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,3
1,0,0,0,-1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,0
2,-1,0,0,-1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,2
3,-1,0,-1,-1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3


In [11]:
board_df.to_csv('board_data.csv', index=False)

In [12]:
import pandas as pd
import numpy as np

data = pd.read_csv('board_data.csv')
data = data.groupby(data.columns.tolist(),as_index=False).size()
data['total_0'] = np.where(data['nextmove column'] == 0, data['size'], 0)
data['total_1'] = np.where(data['nextmove column'] == 1, data['size'], 0)
data['total_2'] = np.where(data['nextmove column'] == 2, data['size'], 0)
data['total_3'] = np.where(data['nextmove column'] == 3, data['size'], 0)
data['total_4'] = np.where(data['nextmove column'] == 4, data['size'], 0)
data['total_5'] = np.where(data['nextmove column'] == 5, data['size'], 0)
data['total_6'] = np.where(data['nextmove column'] == 6, data['size'], 0)
data.head(3)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,player,nextmove column,size,total_0,total_1,total_2,total_3,total_4,total_5,total_6
0,-1,-1,-1,0,-1,1,-1,-1,1,0,...,-1,3,1,0,0,0,1,0,0,0
1,-1,-1,-1,0,0,-1,1,0,-1,0,...,1,5,1,0,0,0,0,0,1,0
2,-1,-1,-1,0,0,-1,1,0,0,0,...,1,3,1,0,0,0,1,0,0,0


In [13]:
data = data.drop(columns=["nextmove column", "size"])

In [14]:
column_names = []
for i in range(6*7):
    column_names.append("current_" + str(i))
column_names.append("player")

data = data.groupby(column_names,as_index=False).sum()
data.loc[data['total_0'] > 5].head(3)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_40,current_41,player,total_0,total_1,total_2,total_3,total_4,total_5,total_6
4962,-1,0,0,0,0,0,0,0,0,0,...,0,0,1,7,11,16,41,15,10,21
15530,0,-1,0,0,0,0,0,0,0,0,...,0,0,1,7,10,21,58,32,24,22
21771,0,0,-1,0,0,0,0,0,0,0,...,0,0,1,12,17,24,97,41,31,18


In [15]:
data['total'] = data[['total_0', 'total_1', 'total_2', 'total_3', 'total_4', 'total_5', 'total_6']].sum(axis=1)
data.loc[data['total'] > 5].head(3)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_41,player,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total
1211,-1,-1,0,1,0,0,0,0,0,0,...,0,1,1,1,1,2,1,1,0,7
3569,-1,0,-1,1,-1,0,1,0,0,0,...,0,1,0,0,2,1,3,1,0,7
3994,-1,0,-1,1,0,0,0,0,0,0,...,0,1,1,0,3,2,2,0,5,13


In [16]:
data['total_0'] = data['total_0'].div(data['total'])
data['total_1'] = data['total_1'].div(data['total'])
data['total_2'] = data['total_2'].div(data['total'])
data['total_3'] = data['total_3'].div(data['total'])
data['total_4'] = data['total_4'].div(data['total'])
data['total_5'] = data['total_5'].div(data['total'])
data['total_6'] = data['total_6'].div(data['total'])
data.loc[data['total'] > 5].head(10)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_41,player,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total
1211,-1,-1,0,1,0,0,0,0,0,0,...,0,1,0.142857,0.142857,0.142857,0.285714,0.142857,0.142857,0.000000,7
3569,-1,0,-1,1,-1,0,1,0,0,0,...,0,1,0.000000,0.000000,0.285714,0.142857,0.428571,0.142857,0.000000,7
3994,-1,0,-1,1,0,0,0,0,0,0,...,0,1,0.076923,0.000000,0.230769,0.153846,0.153846,0.000000,0.384615,13
4389,-1,0,-1,1,1,0,0,0,0,0,...,0,1,0.000000,0.000000,0.000000,0.166667,0.166667,0.333333,0.333333,6
4590,-1,0,0,-1,0,0,1,0,0,0,...,0,1,0.000000,0.111111,0.777778,0.000000,0.000000,0.000000,0.111111,9
4607,-1,0,0,-1,0,1,0,0,0,0,...,0,1,0.000000,0.250000,0.250000,0.375000,0.000000,0.125000,0.000000,8
4746,-1,0,0,-1,1,0,0,0,0,0,...,0,1,0.111111,0.111111,0.388889,0.111111,0.166667,0.000000,0.111111,18
4748,-1,0,0,-1,1,0,0,0,0,0,...,0,1,0.000000,0.000000,0.000000,0.000000,0.857143,0.142857,0.000000,7
4835,-1,0,0,-1,1,1,0,0,0,0,...,0,-1,0.000000,0.428571,0.000000,0.000000,0.285714,0.285714,0.000000,7
4836,-1,0,0,-1,1,1,0,0,0,0,...,0,1,0.000000,0.125000,0.250000,0.125000,0.375000,0.125000,0.000000,8


In [17]:
data = data.drop(columns=["total"])
data.to_csv('board_data2.csv', index=False)

# Prepare model

In [18]:
import pandas as pd

data = pd.read_csv('board_data2.csv')
data.head(3)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_40,current_41,player,total_0,total_1,total_2,total_3,total_4,total_5,total_6
0,-1,-1,-1,0,-1,1,-1,-1,1,0,...,1,1,-1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-1,-1,-1,0,0,-1,1,0,-1,0,...,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-1,-1,-1,0,0,-1,1,0,0,0,...,0,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
from sklearn.model_selection import train_test_split
from numpy.random import seed
seed(1746890)
from tensorflow.random import set_seed
set_seed(1746890)

train, test = train_test_split(data, test_size=0.2, random_state=1746890, shuffle=True)

column_names = []
for i in range(6*7):
    column_names.append("current_" + str(i))
column_names.append("player")

X_train = train.drop(columns=["total_0", "total_1", "total_2", "total_3", "total_4", "total_5", "total_6"], axis=1)
X_test = test.drop(columns=["total_0", "total_1", "total_2", "total_3", "total_4", "total_5", "total_6"], axis=1)
Y_train = train.drop(columns=column_names, axis=1)
Y_test = test.drop(columns=column_names, axis=1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(42186, 43)
(10547, 43)
(42186, 7)
(10547, 7)


In [20]:
Y_train.head(3)

,total_0,total_1,total_2,total_3,total_4,total_5,total_6
50078,0.0,0.0,1.0,0.0,0.0,0.0,0.0
49201,0.0,1.0,0.0,0.0,0.0,0.0,0.0
28097,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [21]:
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.values
Y_test = Y_test.values
print(X_train)

[[ 1  1 -1 ...  0  0 -1]
 [ 1  0  1 ...  0  0  1]
 [ 0  0  1 ...  0  0  1]
 ...
 [-1  1  1 ...  0  0 -1]
 [ 0  1  0 ...  0  0 -1]
 [-1  0  0 ...  0  0  1]]


In [22]:
from tensorflow import keras

inputs = keras.Input(shape=X_train.shape[1])
hidden_layer = keras.layers.Dense(128, activation="sigmoid")(inputs)
hidden_layer = keras.layers.Dense(128, activation="sigmoid")(hidden_layer)
output_layer = keras.layers.Dense(7, activation="softmax")(hidden_layer)

model = keras.Model(inputs=inputs, outputs=output_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 43)]              0         
                                                                 
 dense (Dense)               (None, 128)               5632      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 7)                 903       
                                                                 
Total params: 23,047
Trainable params: 23,047
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy(), metrics=['Accuracy'])

In [47]:
history = model.fit(X_train, Y_train, epochs=10)

Epoch 1/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.5708 - Accuracy: 0.4037
Epoch 2/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.5630 - Accuracy: 0.4021
Epoch 3/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.5554 - Accuracy: 0.4089
Epoch 4/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.5495 - Accuracy: 0.4090
Epoch 5/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.5422 - Accuracy: 0.4123
Epoch 6/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.5379 - Accuracy: 0.4145
Epoch 7/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.5315 - Accuracy: 0.4169
Epoch 8/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.5267 - Accuracy: 0.4194
Epoch 9/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.5227 - Accuracy: 0.4203
Epoch 10/10
1319/1319 [==============================] - 4s 3ms/step - loss: 1.516

In [48]:
model.save('model2')

INFO:tensorflow:Assets written to: model2\assets


In [49]:
from tensorflow import keras
pretrained_model = keras.models.load_model('model2')

In [52]:
board = [[0,1,1,1,-1,-1,0],[0,0,-1,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0], [1]]
flat_board = [item for sublist in board for item in sublist]
pred = pretrained_model.predict([flat_board])
print(pred)

[[0.5057988  0.16127409 0.07581603 0.12721406 0.07029299 0.03956339
  0.02004054]]


In [53]:
import numpy as np
pred_column = np.array(pred[0]).argmax()
pred_chance = pred[0][pred_column] * 100
print('column {} with {:.2f}%'.format(pred_column, pred_chance))

column 0 with 50.58%
